In [1]:
#### 
# The purpose of this notebok is to estimate h, epsilon and theta based on the actual values in the synthetic
# dataset. For this purpose we read the gt heterozygous loci positions, then we read the gt mutation positions for
# each of the tumor groups (1 to 8), then we read the "significant positiosns" writend by SECEDO (i.e. positions that
# passed the Bayesian filtering). This is enough to estimate the percentage of homozygous loci (h) and mutated loci 
# (epsilon)
# The estimation of theta (seq error rate) is trickier, as we don't have the gt from Art Illumina. We instead stimate
# the sequencing error rate by reading the pileup files and counting positions where the homozygous loci have reads 
# different than the expected base.
####


# read all heterozygous loci in the healthy genome

base_dir='/cluster/home/ddanciu/somatic_variant_calling/varsim_data/genomes/'

hetero_pos = []
# read all heterozygous positions in the healthy VCF file
healthy_vcf = 'healthy.truth.vcf'
fname = f'{base_dir}{healthy_vcf}'
print(f'Processing {fname}')
f = open(fname)
for line in f:
    if line.startswith('#'):
         continue
    cols = line.split('\t')
    # parse the last column, indicating if a cell is homo ref/alt or heterozygous
    last_col = cols[-1].split('|')
    if len(last_col) != 2:  # X or Y chromozome
        continue
    if int(last_col[0]) != int(last_col[1]):
        hetero_pos.append((cols[0], int(cols[1])))
print(f'Number of heterozyougs positions in healthy genome: {len(hetero_pos)}')

Processing /cluster/home/ddanciu/somatic_variant_calling/varsim_data/genomes/healthy.truth.vcf
Number of heterozyougs positions in healthy genome: 1870674


In [3]:
tumor_vcfs=['tumor1-h-20K/tumor1-h-20K_somatic.vcf','tumor2-1-10K/tumor2-1-10K_somatic.vcf', 'tumor3-1-15K/tumor3-1-15K_somatic.vcf',
'tumor4-3-10K/tumor4-3-10K_somatic.vcf', 'tumor5-3-5K/tumor5-3-5K_somatic.vcf','tumor6-3-7.5K/tumor6-3-7.5K_somatic.vcf', 
'tumor7-1-5K/tumor7-1-5K_somatic.vcf','tumor8-5-2.5K/tumor8-5-2.5K_somatic.vcf']
#'tumor10-h-0.75K/tumor10-h-0.75K_somatic.vcf', 'tumor9-h-1.25K/tumor9-h-1.25K_somatic.vcf']

out_dir = '/cluster/work/grlab/projects/projects2019-secedo/datasets/varsim/big/secedo_5_01/'

# here we read the ground truth mutation positions for all tumors
mutated_pos = []
for tumor_vcf in tumor_vcfs:
    fname = f'{base_dir}{tumor_vcf}'
    print(f'Processing {fname}')
    f = open(fname)
    for line in f:
        if line.startswith('#'):
             continue
        cols = line.split('\t')
        mutated_pos.append((cols[0], int(cols[1])))  # (chromosome, position)
    print(f'Number of positions: {len(mutated_pos)}')

# list of the significant positions that correspond to non-homozygous loci (one list per clustering tree node)
not_homo_significant_list = []
significant_list = []
# here we read the significant positions as computed by SECEDO
mutation_ratios = []
homo_ratios = []
hetero_ratios = []
for suffix in ['', 'A', 'B', 'AA', 'BA', 'BB']:
    fname = f'{out_dir}significant_positions{suffix}'
    print(f'Processing significant positions from {fname}')
    significant_pos = []
    f = open(fname)
    for line in f:
        if line.startswith('#'):
             continue
        cols = line.split('\t')
        significant_pos.append((cols[0], int(cols[1])))
    print(f'Number of significant positions: {len(significant_pos)}')

    hetero_significant = set(hetero_pos) & set(significant_pos)
    mutated_significant = set(mutated_pos) & set(significant_pos)
    hetero_or_mutated_significant = (set(hetero_pos)|set(mutated_pos)) & set(significant_pos)
    not_homo_significant_list.append(hetero_or_mutated_significant)
    significant_list.append(significant_pos)
    
    hetero_ratios.append(len(hetero_significant)/len(significant_pos))
    mutation_ratios.append(len(mutated_significant)/len(significant_pos))
    homo_ratios.append((len(significant_pos)-len(hetero_or_mutated_significant))/len(significant_pos))

print(f'Heterozygous ratios: {hetero_ratios}')
print(f'Mutations ratios: {mutation_ratios}')
print(f'Homo ratios: {homo_ratios}')


Processing /cluster/home/ddanciu/somatic_variant_calling/varsim_data/genomes/tumor1-h-20K/tumor1-h-20K_somatic.vcf
Number of positions: 20868
Processing /cluster/home/ddanciu/somatic_variant_calling/varsim_data/genomes/tumor2-1-10K/tumor2-1-10K_somatic.vcf
Number of positions: 52554
Processing /cluster/home/ddanciu/somatic_variant_calling/varsim_data/genomes/tumor3-1-15K/tumor3-1-15K_somatic.vcf
Number of positions: 89248
Processing /cluster/home/ddanciu/somatic_variant_calling/varsim_data/genomes/tumor4-3-10K/tumor4-3-10K_somatic.vcf
Number of positions: 136733
Processing /cluster/home/ddanciu/somatic_variant_calling/varsim_data/genomes/tumor5-3-5K/tumor5-3-5K_somatic.vcf
Number of positions: 179225
Processing /cluster/home/ddanciu/somatic_variant_calling/varsim_data/genomes/tumor6-3-7.5K/tumor6-3-7.5K_somatic.vcf
Number of positions: 224232
Processing /cluster/home/ddanciu/somatic_variant_calling/varsim_data/genomes/tumor7-1-5K/tumor7-1-5K_somatic.vcf
Number of positions: 250934
Proc

In [5]:
# Estimate theta (sequencing error rate) by counting homozygous signficant (passing the Bayesian filter) positions 
# that have different bases.

# In order to run this notebook, you need (in addition to the files available in this repository), the pileup files
# generated by Secedo (~11GB, too large to submit here)


import struct


chromosomes = list(range(1,23)) + ['X']
chromosomes = [str(s) for s in chromosomes]
error_rates = []
for idx, not_homo in enumerate(not_homo_significant_list):
    data_count = 0
    m_idx = 0
    c_idx = 0
    c = '2'
    # parsing all data is too slow; pick a random chromosome and a subset of loci from it
    fname=f'/cluster/home/ddanciu/somatic_variant_calling/varsim_data/big/pileups/chromosome_2.pileup.bin'
    outfname=f'/cluster/home/ddanciu/somatic_variant_calling/varsim_data/big/pileups/chromosome_2.pileup.pos'
    ff = open(outfname, 'w')
    f = open(fname, 'rb')
    same = 0
    mutation_count = 0
    total_count = 0
    while True:
        rbytes = f.read(4)
        if rbytes == b"":
            break

        pos, = struct.unpack('I', rbytes)
        coverage, = struct.unpack('H', f.read(2))
        f.read(4 * coverage)  # read ids
        
        cell_id_bases = []
        
        if ((c, pos) not in not_homo and (c,pos) in significant_list[idx]): # homozygous and significant
            data_count += 1
            counts = [0,0,0,0]
            for i in range(coverage):
                cell_id_base, = struct.unpack('H', f.read(2))
                base = cell_id_base & 3
                cell_id = cell_id_base >> 2
                cell_id_bases.append((cell_id, base))
                counts[base] += 1
            old_counts = counts.copy()
            counts.sort()
            
            # print(pos, coverage, counts[3])

            # due to a bug in Varsim (or Bowtie misalignments?), some clearly heterozygous loci 
            # seep through, as they are not marked in the VCF; if more than 8% of the bases differ, 
            # we are clearly not dealing with a homozygous locus, so we eliminate it
            if coverage-counts[3] < 0.08 * coverage:
                mutation_count += coverage - counts[3]
                total_count += coverage
                
            if coverage-counts[3] > 0.2 * coverage:
                s = ' '.join([str(i) for i in [c, pos, old_counts[0], old_counts[1],  old_counts[2], old_counts[3]]])
                ff.write(s)
                ff.write('\n')
                         
            
            if data_count % 10 == 0:
                ff.flush()
                print('.', end='')
        else:
            f.read(2*coverage)
            
        #if data_count > 1e2:
        #    break
        


    error_rates.append(0 if total_count==0 else mutation_count/total_count)
    print(error_rates)
    break

print(f'Error estimate based on 10K position: {error_rates}')
        
    
    
    

Testing node  0
...................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[0.05668576578896543]
Error estimate based on 10K position: [0.05668576578896543]


In [ ]:
# Justify why we eliminate loci with c-c1<5, by counting the number of mutated loci for c-c1=1,2,3,4,5,etc.


import struct
import matplotlib.pyplot as plt
import numpy as np


error_rates = []
# select the positions for the first node of the clustering tree
not_homo = not_homo_significant_list[0]

# parsing all data is too slow; pick a random chromosome and a subset of loci from it
c='8'
fname=f'/cluster/home/ddanciu/somatic_variant_calling/varsim_data/big/pileups/chromosome_{c}.pileup.bin'
f = open(fname, 'rb')

gt_mutations = set([m for m in mutated_pos if m[0]==c])
print(f'Loaded {len(gt_mutations)} mutated positions for chromosome {c}')


all_loci = [0] * 40
mutated_loci = [0] * 40
idx = 0
while True:
    rbytes = f.read(4)
    if rbytes == b"":
        break

    pos, = struct.unpack('I', rbytes)
    coverage, = struct.unpack('H', f.read(2))
    f.read(4 * coverage)  # read ids

    cell_id_bases = []
    counts = [0,0,0,0]
    for i in range(coverage):
        cell_id_base, = struct.unpack('H', f.read(2))
        base = cell_id_base & 3
        cell_id = cell_id_base >> 2
        cell_id_bases.append((cell_id, base))
        counts[base] += 1
    counts.sort()
    diff = coverage - counts[3]
    idx += 1
    if diff < 40:
        all_loci[diff] += 1
        if (c, pos) in gt_mutations:
            mutated_loci[diff] += 1

    if idx % 100000 == 0:
        print(f'Total positions for c-c1: {all_loci}')
        print(f'Mutated positions for c-c1: {mutated_loci}')



num_pos = 21

fig, axs = plt.subplots(1, figsize=(9,6))

ratios=[m/a if a!=0 else 0 for (m,a) in zip(mutated_loci, all_loci)][:num_pos]
labels = range(num_pos)

 
axs.bar(labels, ratios, color='blue', alpha = 0.8)
plt.ylabel("Mutated loci", fontsize=14)
axs.set_xlim([0,20.5])
plt.xticks(range(num_pos), fontsize=14)
plt.yticks(fontsize=14)
plt.savefig('/tmp/mutated_loci_by_diff.png', format='png', bbox_inches='tight')



Loaded 4366 mutated positions for chromosome 8
Total positions for c-c1: [97898, 1959, 81, 4, 2, 4, 3, 2, 5, 1, 1, 1, 1, 2, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0]
Mutated positions for c-c1: [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Total positions for c-c1: [193234, 6338, 270, 12, 4, 4, 4, 2, 6, 2, 1, 1, 1, 2, 1, 0, 0, 0, 0, 2, 1, 1, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 1, 0]
Mutated positions for c-c1: [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Total positions for c-c1: [290647, 8754, 359, 15, 8, 5, 10, 3, 8, 3, 2, 4, 2, 2, 3, 0, 0, 2, 1, 2, 3, 1, 2, 1, 0, 0, 0, 0, 0, 1, 1, 0, 3, 1, 0, 1, 1, 1, 2, 0]
Mutated positions for c-c1: [3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Total positions for c-c1: [383014,

Total positions for c-c1: [2520344, 168051, 6835, 321, 97, 69, 64, 61, 53, 51, 25, 30, 21, 23, 27, 16, 15, 15, 12, 13, 13, 6, 11, 11, 2, 5, 2, 0, 4, 1, 3, 5, 5, 5, 5, 3, 2, 1, 6, 1]
Mutated positions for c-c1: [15, 1, 1, 0, 1, 0, 2, 4, 3, 5, 3, 10, 3, 4, 6, 3, 1, 0, 0, 4, 2, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Total positions for c-c1: [2612874, 175088, 7112, 331, 103, 71, 67, 62, 54, 52, 27, 31, 21, 23, 27, 18, 16, 16, 12, 13, 13, 7, 11, 11, 2, 5, 2, 1, 5, 2, 4, 5, 5, 6, 5, 3, 2, 1, 6, 2]
Mutated positions for c-c1: [15, 1, 1, 0, 2, 0, 2, 5, 3, 5, 3, 10, 3, 4, 6, 3, 1, 0, 0, 4, 2, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Total positions for c-c1: [2704983, 182487, 7406, 341, 104, 76, 71, 63, 57, 55, 30, 31, 21, 26, 28, 18, 17, 16, 13, 14, 13, 7, 11, 11, 2, 5, 3, 1, 5, 2, 4, 5, 5, 6, 5, 3, 2, 1, 6, 2]
Mutated positions for c-c1: [15, 1, 1, 0, 2, 0, 2, 5, 3, 5, 4, 10, 3, 4, 6, 3, 1, 0, 0, 5, 2, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

Total positions for c-c1: [4881431, 298468, 12119, 545, 179, 118, 123, 97, 94, 85, 55, 54, 41, 37, 38, 29, 32, 22, 21, 23, 15, 13, 14, 11, 8, 6, 5, 1, 10, 5, 6, 7, 7, 8, 5, 4, 5, 2, 6, 2]
Mutated positions for c-c1: [30, 2, 2, 2, 2, 1, 2, 6, 6, 7, 6, 11, 6, 6, 7, 4, 4, 0, 2, 5, 2, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
Total positions for c-c1: [4981087, 298768, 12128, 554, 185, 121, 127, 98, 98, 85, 56, 54, 42, 37, 38, 30, 32, 22, 21, 23, 15, 13, 14, 11, 8, 7, 5, 1, 10, 5, 6, 7, 7, 8, 5, 4, 5, 2, 6, 2]
Mutated positions for c-c1: [32, 2, 2, 2, 2, 1, 3, 6, 6, 7, 6, 11, 6, 6, 7, 4, 4, 0, 2, 5, 2, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
Total positions for c-c1: [5081017, 298835, 12130, 554, 185, 122, 127, 98, 98, 85, 56, 54, 42, 37, 38, 30, 32, 22, 21, 23, 15, 13, 14, 11, 8, 7, 5, 1, 10, 5, 6, 7, 7, 8, 5, 4, 5, 2, 6, 2]
Mutated positions for c-c1: [32, 2, 2, 2, 2, 1, 3, 6, 6, 7, 6, 11, 6, 6, 7, 4, 4, 0, 2, 5, 2, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0

Total positions for c-c1: [7127176, 443722, 18148, 791, 250, 183, 182, 141, 134, 120, 83, 75, 70, 58, 62, 41, 39, 29, 27, 28, 21, 17, 18, 15, 13, 7, 6, 4, 12, 6, 10, 8, 7, 11, 5, 5, 6, 2, 7, 3]
Mutated positions for c-c1: [41, 3, 3, 3, 2, 2, 4, 10, 9, 13, 8, 15, 9, 13, 11, 4, 5, 1, 3, 5, 2, 1, 1, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0]
Total positions for c-c1: [7219430, 451061, 18423, 805, 255, 185, 184, 141, 137, 121, 86, 76, 70, 58, 67, 41, 40, 30, 27, 29, 21, 18, 18, 15, 13, 7, 6, 4, 12, 6, 10, 8, 8, 11, 5, 5, 6, 2, 7, 3]
Mutated positions for c-c1: [41, 3, 3, 3, 2, 2, 4, 10, 9, 13, 8, 16, 9, 13, 11, 4, 5, 1, 3, 5, 2, 1, 1, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0]
Total positions for c-c1: [7311696, 458329, 18730, 820, 258, 189, 187, 145, 139, 125, 89, 78, 73, 59, 67, 41, 40, 30, 27, 30, 21, 18, 19, 15, 13, 7, 6, 4, 12, 6, 10, 8, 8, 11, 5, 5, 6, 2, 7, 3]
Mutated positions for c-c1: [41, 3, 3, 3, 2, 2, 4, 10, 9, 13, 8, 16, 10, 13, 11, 4, 5, 1, 3, 5, 2, 1, 2, 2, 

Total positions for c-c1: [9345204, 615322, 25111, 1103, 357, 265, 244, 199, 191, 153, 125, 108, 101, 71, 87, 59, 52, 45, 37, 36, 29, 25, 24, 21, 17, 8, 9, 8, 14, 6, 12, 11, 9, 14, 8, 7, 8, 6, 8, 5]
Mutated positions for c-c1: [45, 3, 3, 3, 3, 3, 6, 11, 11, 16, 10, 17, 11, 17, 13, 5, 7, 1, 3, 5, 4, 2, 3, 3, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1]
Total positions for c-c1: [9437444, 622619, 25396, 1123, 363, 270, 246, 201, 193, 155, 127, 110, 101, 71, 87, 61, 52, 45, 37, 36, 29, 25, 24, 21, 17, 8, 9, 8, 14, 6, 12, 12, 9, 14, 8, 7, 8, 6, 8, 5]
Mutated positions for c-c1: [45, 3, 3, 3, 3, 3, 6, 11, 12, 16, 10, 17, 11, 17, 13, 6, 7, 1, 3, 5, 4, 2, 3, 3, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1]
Total positions for c-c1: [9529583, 630023, 25716, 1133, 370, 271, 247, 202, 194, 155, 129, 110, 102, 71, 87, 61, 52, 45, 37, 36, 29, 25, 24, 21, 17, 8, 9, 8, 14, 6, 12, 12, 9, 14, 8, 7, 8, 6, 8, 5]
Mutated positions for c-c1: [45, 3, 3, 3, 3, 3, 6, 11, 12, 16, 10, 17, 11, 17, 13, 6, 7

...

Total positions for c-c1: [38176503, 2767049, 112086, 4829, 1435, 1119, 892, 772, 659, 533, 471, 402, 359, 268, 270, 181, 191, 147, 132, 108, 90, 95, 85, 80, 62, 46, 39, 39, 42, 30, 35, 31, 28, 32, 20, 27, 28, 23, 29, 23]
Mutated positions for c-c1: [106, 17, 6, 5, 10, 13, 20, 28, 33, 48, 43, 43, 37, 45, 37, 24, 25, 11, 12, 12, 8, 7, 12, 5, 6, 2, 4, 3, 3, 0, 3, 0, 1, 3, 1, 4, 1, 4, 5, 4]
Total positions for c-c1: [38269526, 2773665, 112341, 4840, 1439, 1119, 892, 774, 664, 533, 472, 404, 361, 269, 270, 182, 191, 147, 132, 108, 91, 95, 85, 80, 63, 46, 39, 39, 42, 30, 35, 32, 28, 32, 21, 27, 28, 23, 29, 23]
Mutated positions for c-c1: [106, 17, 6, 5, 10, 13, 20, 28, 33, 48, 43, 43, 37, 46, 37, 25, 25, 11, 12, 12, 8, 7, 12, 5, 7, 2, 4, 3, 3, 0, 3, 0, 1, 3, 2, 4, 1, 4, 5, 4]
Total positions for c-c1: [38361641, 2781120, 112678, 4849, 1442, 1119, 898, 774, 665, 534, 474, 405, 362, 272, 271, 182, 191, 147, 132, 108, 91, 95, 85, 81, 63, 47, 39, 39, 42, 31, 35, 32, 28, 33, 21, 27, 28, 23, 29, 

Total positions for c-c1: [40119199, 2916128, 118181, 5065, 1495, 1175, 943, 813, 692, 561, 497, 428, 382, 288, 283, 191, 199, 156, 136, 113, 94, 99, 86, 82, 67, 50, 41, 41, 44, 33, 38, 35, 29, 34, 22, 27, 30, 25, 30, 23]
Mutated positions for c-c1: [109, 19, 6, 5, 10, 14, 21, 29, 33, 52, 47, 46, 39, 47, 38, 25, 26, 12, 12, 12, 8, 8, 12, 5, 7, 2, 4, 4, 3, 0, 3, 0, 1, 3, 2, 4, 1, 4, 5, 4]
Total positions for c-c1: [40211784, 2923187, 118469, 5072, 1496, 1175, 944, 813, 693, 561, 498, 429, 382, 288, 283, 191, 199, 157, 136, 113, 94, 99, 86, 82, 67, 50, 41, 41, 44, 33, 38, 35, 29, 34, 22, 27, 30, 25, 30, 23]
Mutated positions for c-c1: [109, 19, 6, 5, 10, 14, 21, 29, 33, 52, 48, 46, 39, 47, 38, 25, 26, 13, 12, 12, 8, 8, 12, 5, 7, 2, 4, 4, 3, 0, 3, 0, 1, 3, 2, 4, 1, 4, 5, 4]
Total positions for c-c1: [40304663, 2929877, 118773, 5089, 1503, 1182, 950, 815, 697, 565, 498, 430, 384, 290, 285, 191, 200, 158, 136, 113, 95, 100, 86, 82, 67, 50, 41, 42, 46, 33, 39, 35, 30, 34, 22, 27, 30, 25, 30,

Total positions for c-c1: [42066430, 3060948, 124091, 5300, 1568, 1230, 983, 849, 719, 593, 519, 453, 396, 299, 297, 199, 208, 166, 144, 118, 103, 106, 88, 88, 71, 51, 42, 42, 47, 38, 42, 36, 30, 34, 23, 27, 31, 26, 31, 23]
Mutated positions for c-c1: [114, 22, 6, 5, 10, 14, 21, 30, 35, 52, 53, 48, 39, 49, 40, 26, 28, 13, 13, 12, 8, 8, 12, 5, 7, 2, 5, 4, 3, 0, 3, 0, 1, 3, 2, 4, 2, 4, 5, 4]
Total positions for c-c1: [42159074, 3067915, 124380, 5309, 1571, 1230, 983, 849, 719, 594, 519, 455, 396, 299, 297, 199, 209, 166, 145, 118, 103, 107, 88, 89, 71, 51, 42, 42, 47, 38, 42, 36, 30, 34, 23, 27, 31, 26, 31, 23]
Mutated positions for c-c1: [114, 22, 6, 6, 10, 14, 21, 30, 35, 52, 53, 49, 39, 49, 40, 26, 28, 13, 13, 12, 8, 8, 12, 6, 7, 2, 5, 4, 3, 0, 3, 0, 1, 3, 2, 4, 2, 4, 5, 4]
Total positions for c-c1: [42251735, 3074873, 124651, 5319, 1578, 1234, 989, 852, 722, 596, 520, 456, 396, 301, 297, 199, 209, 166, 145, 119, 103, 107, 88, 89, 71, 51, 42, 42, 47, 38, 42, 36, 30, 34, 23, 27, 31, 26

Total positions for c-c1: [44011452, 3208017, 130010, 5494, 1639, 1277, 1027, 889, 749, 625, 542, 475, 407, 310, 302, 205, 215, 171, 150, 126, 107, 109, 90, 89, 74, 51, 45, 42, 49, 39, 44, 37, 32, 36, 25, 30, 31, 26, 31, 26]
Mutated positions for c-c1: [115, 23, 6, 6, 10, 15, 23, 33, 40, 54, 54, 50, 40, 50, 40, 27, 28, 15, 14, 12, 8, 8, 12, 6, 7, 2, 5, 4, 3, 0, 3, 0, 1, 3, 2, 5, 2, 4, 5, 4]
Total positions for c-c1: [44104518, 3214619, 130258, 5501, 1643, 1278, 1027, 889, 749, 626, 542, 477, 408, 311, 303, 207, 217, 171, 150, 126, 107, 110, 90, 89, 74, 51, 45, 42, 49, 39, 44, 37, 32, 36, 25, 30, 31, 26, 31, 26]
Mutated positions for c-c1: [115, 23, 6, 6, 10, 15, 23, 33, 40, 54, 54, 50, 40, 50, 40, 27, 28, 15, 14, 12, 8, 8, 12, 6, 7, 2, 5, 4, 3, 0, 3, 0, 1, 3, 2, 5, 2, 4, 5, 4]
Total positions for c-c1: [44196908, 3221823, 130530, 5524, 1646, 1282, 1031, 894, 752, 630, 546, 478, 409, 311, 305, 208, 217, 171, 151, 128, 107, 110, 93, 89, 74, 51, 45, 42, 49, 40, 45, 37, 32, 36, 25, 30, 31,

Total positions for c-c1: [45953827, 3357648, 136014, 5725, 1697, 1327, 1068, 930, 780, 656, 565, 494, 426, 319, 311, 221, 224, 178, 156, 135, 117, 112, 100, 91, 77, 52, 46, 42, 50, 45, 46, 37, 33, 38, 25, 30, 32, 26, 31, 26]
Mutated positions for c-c1: [117, 23, 6, 6, 10, 17, 25, 33, 43, 56, 55, 50, 43, 51, 42, 30, 29, 16, 14, 12, 8, 8, 13, 7, 7, 2, 5, 4, 3, 1, 3, 0, 1, 3, 2, 5, 2, 4, 5, 4]
Total positions for c-c1: [46046959, 3364129, 136298, 5735, 1700, 1328, 1068, 931, 782, 658, 571, 495, 426, 320, 312, 221, 225, 178, 156, 137, 117, 114, 100, 92, 77, 53, 47, 42, 51, 46, 46, 38, 34, 40, 25, 30, 33, 26, 31, 26]
Mutated positions for c-c1: [117, 23, 6, 7, 10, 17, 25, 33, 43, 56, 56, 50, 43, 51, 42, 30, 29, 16, 14, 12, 8, 8, 13, 7, 7, 2, 5, 4, 3, 1, 3, 0, 1, 3, 2, 5, 2, 4, 5, 4]
Total positions for c-c1: [46139942, 3370791, 136575, 5742, 1702, 1329, 1070, 931, 784, 658, 571, 495, 426, 320, 312, 221, 225, 178, 156, 137, 117, 114, 100, 92, 77, 53, 47, 42, 52, 46, 46, 38, 35, 40, 25, 30, 